In [1]:
!pip install torch torchvision opencv-python numpy segmentation-models-pytorch piq

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 11.1 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=2c145aa6bd6e1ce5199a41002a7e8593c9545c91c49468a4c5f180a653fcb57c
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=6747bf25790c48915c1c5cc700b1cefa2890256fe61634765229d43cc84a04ba
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60

In [2]:
!pip install segmentation-models-pytorch

In [4]:
# Install the Kaggle API client
!pip install kaggle

# Create the directory for storing the Kaggle API key
import os
os.makedirs('/root/.kaggle', exist_ok=True)

# Move the uploaded kaggle.json file to the correct directory
import shutil
shutil.move('/content/kaggle.json', '/root/.kaggle/kaggle.json')

# Set the permission for the kaggle.json file
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d awsaf49/kitti-eigen-split-dataset


Dataset URL: https://www.kaggle.com/datasets/awsaf49/kitti-eigen-split-dataset
License(s): CC0-1.0
100% 21.7G/21.7G [02:56<00:00, 166MB/s]
100% 21.7G/21.7G [02:56<00:00, 132MB/s]


In [6]:
!unzip /content/kitti-eigen-split-dataset.zip -d /content/kitti-eigen-split-dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/kitti-eigen-split-dataset/train/2011_10_03/2011_10_03_drive_0034_sync/proj_depth/groundtruth/image_02/0000001978.png  
  inflating: /content/kitti-eigen-split-dataset/train/2011_10_03/2011_10_03_drive_0034_sync/proj_depth/groundtruth/image_02/0000001979.png  
  inflating: /content/kitti-eigen-split-dataset/train/2011_10_03/2011_10_03_drive_0034_sync/proj_depth/groundtruth/image_02/0000001980.png  
  inflating: /content/kitti-eigen-split-dataset/train/2011_10_03/2011_10_03_drive_0034_sync/proj_depth/groundtruth/image_02/0000001981.png  
  inflating: /content/kitti-eigen-split-dataset/train/2011_10_03/2011_10_03_drive_0034_sync/proj_depth/groundtruth/image_02/0000001982.png  
  inflating: /content/kitti-eigen-split-dataset/train/2011_10_03/2011_10_03_drive_0034_sync/proj_depth/groundtruth/image_02/0000001983.png  
  inflating: /content/kitti-eigen-split-dataset/train/2011_10_03/2011_10_03_drive_0034_sync/proj_depth/

In [17]:
import os

# Path to KITTI Eigen Split dataset
base_path = '/content/kitti-eigen-split-dataset'
train_data_path = os.path.join(base_path, 'train')


In [18]:
from torch.utils.data import Dataset # Import Dataset from torch.utils.data

class KITTIDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = []
        self.depth_files = []

        # Traverse directories to find images and groundtruth depth maps
        for date_folder in os.listdir(root_dir):
            date_path = os.path.join(root_dir, date_folder)
            if os.path.isdir(date_path):
                for drive_folder in os.listdir(date_path):
                    drive_path = os.path.join(date_path, drive_folder)
                    image_dir = os.path.join(drive_path, 'image_02/data')
                    depth_dir = os.path.join(drive_path, 'proj_depth/groundtruth/image_02')

                    if os.path.exists(image_dir) and os.path.exists(depth_dir):
                        images = sorted(os.listdir(image_dir))
                        depths = sorted(os.listdir(depth_dir))

                        for img, depth in zip(images, depths):
                            image_file = os.path.join(image_dir, img)
                            depth_file = os.path.join(depth_dir, depth)

                            if os.path.isfile(image_file) and os.path.isfile(depth_file):
                                self.image_files.append(image_file)
                                self.depth_files.append(depth_file)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        depth_path = self.depth_files[idx]

        image = Image.open(image_path).convert('RGB')
        depth = Image.open(depth_path).convert('L')  # Grayscale depth map

        if self.transform:
            image = self.transform(image)
            depth = self.transform(depth)

        return image, depth

In [19]:
# Import the necessary module
from torchvision import transforms

#TRANSFORMATIONS
transform = transforms.Compose([
    transforms.Resize((256, 512)),
    transforms.ToTensor(),
])

In [20]:
#Initialize Dataset and DataLoader
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

train_dataset = KITTIDataset(train_data_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

print(f"Total training samples: {len(train_dataset)}")

Total training samples: 23158


In [21]:
#ENCODER

import torchvision.models as models
import torch.nn as nn # Import the torch.nn module


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        resnet = models.resnet50(weights='IMAGENET1K_V1')
        self.initial_layers = nn.Sequential(*list(resnet.children())[:4])
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4

    def forward(self, x):
        x = self.initial_layers(x)
        x1 = self.layer1(x)
        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        x4 = self.layer4(x3)
        return x1, x2, x3, x4

In [22]:
import torch.nn.functional as F


class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.upconv4 = nn.ConvTranspose2d(2048, 1024, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.iconv4 = nn.Conv2d(1024 + 1024, 1024, kernel_size=3, stride=1, padding=1)
        self.depth_pred = nn.Conv2d(1024, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, x1, x2, x3, x4):
        up4 = self.upconv4(x4)
        concat4 = torch.cat((up4, x3), dim=1)
        iconv4 = self.iconv4(concat4)

        # Predict depth
        depth = self.depth_pred(iconv4)

        # Upsample to match ground truth size
        depth = F.interpolate(depth, size=(256, 512), mode='bilinear', align_corners=False)
        return depth



In [23]:
#Full Model
import torch
import torch.nn as nn

# Define the U-Net architecture
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.inc = self.double_conv(n_channels, 64)
        self.down1 = self.down(64, 128)
        self.down2 = self.down(128, 256)
        self.down3 = self.down(256, 512)
        self.down4 = self.down(512, 512)
        self.up1 = self.up(1024, 256)
        self.up2 = self.up(512, 128)
        self.up3 = self.up(256, 64)
        self.up4 = self.up(128, 64)
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)

    def double_conv(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def down(self, in_channels, out_channels):
        return nn.Sequential(
            nn.MaxPool2d(2),
            self.double_conv(in_channels, out_channels)
        )

    def up(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2),
            self.double_conv(in_channels, out_channels)
        )

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(torch.cat([x4, x5], dim=1))
        x = self.up2(torch.cat([x3, x], dim=1))
        x = self.up3(torch.cat([x2, x], dim=1))
        x = self.up4(torch.cat([x1, x], dim=1))
        logits = self.outc(x)
        return logits

# Initialize the U-Net model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet(n_channels=3, n_classes=1).to(device)
print("U-Net model initialized:", model)


U-Net model initialized: UNet(
  (inc): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (down1): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down2): Sequential(

In [24]:
#Training set up

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DepthEstimationModel().to(device)

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [25]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (images, depths) in enumerate(train_loader):
        images, depths = images.to(device), depths.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Compute loss
        loss = criterion(outputs, depths)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 50 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}], Loss: {running_loss / 50:.4f}")
            running_loss = 0.0

print("Training complete.")


Epoch [1/10], Step [50], Loss: 0.2542
Epoch [1/10], Step [100], Loss: 0.1304
Epoch [1/10], Step [150], Loss: 0.1227
Epoch [1/10], Step [200], Loss: 0.1168
Epoch [1/10], Step [250], Loss: 0.1167
Epoch [1/10], Step [300], Loss: 0.1148
Epoch [1/10], Step [350], Loss: 0.1131
Epoch [1/10], Step [400], Loss: 0.1122
Epoch [1/10], Step [450], Loss: 0.1123
Epoch [1/10], Step [500], Loss: 0.1106
Epoch [1/10], Step [550], Loss: 0.1114
Epoch [1/10], Step [600], Loss: 0.1100
Epoch [1/10], Step [650], Loss: 0.1100
Epoch [1/10], Step [700], Loss: 0.1084
Epoch [1/10], Step [750], Loss: 0.1080
Epoch [1/10], Step [800], Loss: 0.1076
Epoch [1/10], Step [850], Loss: 0.1102
Epoch [1/10], Step [900], Loss: 0.1074
Epoch [1/10], Step [950], Loss: 0.1085
Epoch [1/10], Step [1000], Loss: 0.1068
Epoch [1/10], Step [1050], Loss: 0.1068
Epoch [1/10], Step [1100], Loss: 0.1070
Epoch [1/10], Step [1150], Loss: 0.1065
Epoch [1/10], Step [1200], Loss: 0.1061
Epoch [1/10], Step [1250], Loss: 0.1039
Epoch [1/10], Step [

In [26]:
torch.save(model.state_dict(), '/content/kitti-eigen-split-dataset/kitti_UNET_model.pth')
print("Model saved successfully.")

Model saved successfully.
